In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
source_dir = 'midi_npz'

out_dir = 'midi_encode/text/longdur'
continuous = False
short = False
def note_enc(note):
    if continuous: return note.continuous_repr(short=short)
    return note.duration_repr(short=short)

source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'midi_encode.csv'

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,9,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0


In [8]:
df_filtered = df.loc[df[source_dir].notna()]; df_filtered.shape

(38744, 25)

In [9]:
df_filter = (df_filtered.source != 'hooktheory') | (df_filtered.ht_time_signature == 4.0)
df_filtered = df_filtered.loc[df_filter]; df_filtered.shape

(37045, 25)

In [10]:
df_filtered = df_filtered.loc[df_filtered[f'{source_dir}_timesteps'] < 2e4]; df_filtered.shape

(37035, 25)

In [11]:
def transform_func(file, out_file, row):
    chordarr = load_chordarr(file)
    seq = chordarr2seq(chordarr)
    string_repr = seq2str(seq, note_enc, continuous=continuous)
    with open(out_file, 'w') as tf:
        tf.write(string_repr)

In [12]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension='.txt'
       )

In [13]:
for r in df_filtered.iterrows():
    parallel_func(r)
    break

In [14]:
encoded_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

In [15]:
tdf = pd.DataFrame(data={out_dir: list(encoded_files.values())}, index=list(encoded_files.keys()))

In [16]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((37035, 1), (39942, 25), (39942, 26))

In [17]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_encode/text/longdur
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_encode/text/longdur/hooktheory/pianoroll/...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_encode/text/longdur/hooktheory/pianoroll/...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0,midi_encode/text/longdur/hooktheory/pianoroll/...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0,midi_encode/text/longdur/hooktheory/pianoroll/...
